## Glop Optimization with Lists: Sidneyville Manufacturing

In [1]:
# import Glop package
from ortools.linear_solver import pywraplp as glp

In [2]:
# initialize LP model object
mymodel = glp.Solver('Sidneyville', glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [3]:
# input parameters
product = ['Roll_Top', 'Regular']
wood = ['Pine', 'Cedar', 'Maple']
profit = [115.00, 90.00]
wood_req = [[10, 4, 15], [20, 16, 10]]    # each sublist corresponds to a product
wood_rhs = [200, 128, 220]

In [4]:
# define decision variables
dvar = list(range(len(product)))
for i in range(len(product)):
    dvar[i] = mymodel.NumVar(0, mymodel.infinity(), product[i])
    
xtra_var = mymodel.NumVar(0, 0, 'Special')

In [5]:
# define objective function
TotProfit = mymodel.Objective()
TotProfit.SetMaximization()
for i in range(len(product)):
    TotProfit.SetCoefficient(dvar[i], profit[i])

In [6]:
# define constraints
constr = list(range(len(wood)))
for w in range(len(wood)):
    constr[w] = mymodel.Constraint(-mymodel.infinity(), wood_rhs[w], wood[w])
    for p in range(len(product)):
        constr[w].SetCoefficient(dvar[p], wood_req[p][w])

In [12]:
# Print Algebraic Model

infinity = mymodel.infinity()

# Variables
var_lst = mymodel.variables()
var_str = ''
first_var = True
for var in var_lst:
    if first_var:
        var_str = var_str + str(var)
        first_var = False
    else:
        var_str = var_str + ', ' + str(var) 
print('Variables:')
print(var_str, '\n')

# Objective
if mymodel.Objective().maximization():
    obj_str = 'maximize: '
elif mymodel.Objective().minimization():
    obj_str = 'minimize: '
else:
    obj_str = 'undefined: '
    
first_var = True
for var in var_lst:
    coeff = mymodel.Objective().GetCoefficient(var)
    if coeff < 0:
        obj_str = obj_str + '- ' + str(coeff) + '*' + str(var) + ' '
        if first_var: first_var = False
    if first_var and coeff > 0:
        obj_str = obj_str + str(coeff) + '*' + str(var) + ' '
        first_var = False
    elif coeff > 0:
        obj_str = obj_str + '+ ' + str(coeff) + '*' + str(var) + ' '
print(obj_str + '\n')

# Constraints
print('Subject To:')
constr_lst = mymodel.constraints()
for constr in constr_lst:
    constr_str = constr.name() + ': '
    first_var = True
    for var in var_lst:
        coeff = constr.GetCoefficient(var)
        if coeff < 0:
            constr_str = constr_str + '- ' + str(coeff) + '*' + str(var) + ' '
            if first_var: first_var = False
        elif first_var and coeff > 0:
            constr_str = constr_str + str(coeff) + '*' + str(var) + ' '
            first_var = False
        elif coeff > 0:
            constr_str = constr_str + '+ ' + str(coeff) + '*' + str(var) + ' '
    lower = constr.lb()
    upper = constr.ub()
    if lower == upper:
        print(constr_str + '= ' + str(lower))
    elif lower > -infinity and upper == infinity:
        print(constr_str + '>= ' + str(lower))
    elif lower == -infinity and upper < infinity:
        print(constr_str + '<= ' + str(upper))
    elif lower > -infinity and upper < infinity:
        print(constr_str + '>= ' + str(lower))
        print(constr_str + '<= ' + str(upper))
print('')
        
# Variable Bounds
print('Bounds:')
for var in var_lst:
    lower = var.lb()
    upper = var.ub()
    if lower > -infinity and upper == infinity:
        print(str(var) + ' >= ' + str(lower))
    elif lower == -infinity and upper < infinity:
            print(str(var) + ' <= ' + str(upper))
    elif lower > -infinity and upper < infinity:
        print(str(lower) + ' <= ' + str(var) + ' <= ' + str(upper))    

Variables:
Roll_Top, Regular, Special 

maximize: 115.0*Roll_Top + 90.0*Regular 

Subject To:
Pine: 10.0*Roll_Top + 20.0*Regular <= 200.0
Cedar: 4.0*Roll_Top + 16.0*Regular <= 128.0
Maple: 15.0*Roll_Top + 10.0*Regular <= 220.0

Bounds:
Roll_Top >= 0.0
Regular >= 0.0
0.0 <= Special <= 0.0


In [ ]:
# Solve the model and print optimal solution
mymodel.Solve()

print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Profit = %.2f' % TotProfit.Value())

# The value of each variable in the solution.
for p in range(len(product)):
    print('%s = %.2f' % (product[p], dvar[p].solution_value()))

# Dual Values (Shadow Prices)
for w in range(len(wood)):
    print('%s Dual Value = %.2f' % (wood[w], constr[w].dual_value()))